In [1]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from encode_utils.new_mask_utils import randomsingle
from encode_utils.eval_utils import all_lattice_multi, mean, all_unnoun_multi
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

2022-12-20 07:36:10.470730: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 07:36:10.470761: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# run pipeline for generating lattice preds, using different parameters

In [11]:
# set up information for set
gsuffix = "detest_reversed/"
expl_fname = "endeexplodev1.csv"
base = "outputs/graph_pickles/"+gsuffix
goldmetric = "dupcqe"
explode_df = pd.read_csv("outputs/score_csvs/"+expl_fname)
SETLEN = len(os.listdir(base))

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/score_csvs/endeexplodev1.csv'

In [4]:
# use noun model
if "noun" in expl_fname:
    encodemod = get_effrerank_model("noun")
# use mt model (causal)
else:
    encodemod = get_effrerank_model("comstyle")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_43/checkpoints/epoch=3-step=130000.ckpt.


freeze embeds


In [5]:
args = {
    'setlen':SETLEN,
    'tok':xlm_tok, 
    'dev':device,
    'model':encodemod,
    'explode_df':explode_df,
    'base':base,
    'goldmetric':goldmetric,
    'device':device
}

In [ ]:
latpreds = all_lattice_multi(1, default_scofunct, randomsingle, args)

In [8]:
print(mean(latpreds['goldsco']))
print(mean(latpreds['hypsco']))
print(mean(latpreds['goldsco'])-mean(latpreds['hypsco']))

tensor(0.9083, device='cuda:1', grad_fn=<DivBackward0>)
tensor(0.8311, device='cuda:1', grad_fn=<DivBackward0>)
tensor(0.0772, device='cuda:1', grad_fn=<SubBackward0>)


In [47]:
outfile = "enru_comstyle_single_default.csv"
latpreds.to_csv("outputs/predcsvs/"+outfile)

In [10]:
phoc = all_unnoun_multi(-1, "cqe", args)
print(mean(phoc))

tensor(0.7945, device='cuda:1', grad_fn=<DivBackward0>)


In [37]:
explode_df['phocrank'] = explode_df['posthoc']*-1
#rand = all_unnoun_multi(32)

In [32]:
mean(rand)

tensor(0.8749, device='cuda:1', grad_fn=<DivBackward0>)

In [49]:
latpreds[latpreds['goldsco']-latpreds['hypsco']>.2]

,hyp,hypsco,gold,goldsco,ascos,ahyps,numnodes
7,Немецкие и французские стороны явно нуждались ...,"tensor(0.3987, device='cuda:1', grad_fn=<SumBa...",Германия и Франция явно нуждались во взаимном ...,"tensor(0.9256, device='cuda:1', grad_fn=<SumBa...",[0.3987096846103668],[Немецкие и французские стороны явно нуждались...,385
13,Небольшая часть населения Индии не имеет ни од...,"tensor(0.6347, device='cuda:1', grad_fn=<SumBa...",Сложно понять низкую позицию Индии.,"tensor(0.9250, device='cuda:1', grad_fn=<SumBa...",[0.6347033977508545],[Небольшая часть населения Индии не имеет ни о...,501
19,Так много о предсказках Кругмана.,"tensor(0.2668, device='cuda:1', grad_fn=<SumBa...","Это все, что предсказывал Кругман.","tensor(0.7456, device='cuda:1', grad_fn=<SumBa...",[0.2667738199234009],[Так много о предсказках Кругмана.],554
31,"Мы узнали, что это была просто воздушная пузыр...","tensor(0.8288, device='cuda:1', grad_fn=<SumBa...","Мы поняли, что это было просто кружево.","tensor(1.0661, device='cuda:1', grad_fn=<SumBa...",[0.8288499116897583],"[Мы узнали, что это была просто воздушная пузы...",558
71,В некоторых странах до 30% более оптимистичных...,"tensor(0.7622, device='cuda:1', grad_fn=<SumBa...",До 30% наиболее оптимистичных потребителей в н...,"tensor(1.0325, device='cuda:1', grad_fn=<SumBa...",[0.7622228264808655],[В некоторых странах до 30% более оптимистичны...,381
